# Learning Programming with Rust: 101

This document is a guide to help you learn how to think in a functional, data oriented style and move away from
imperative, unprincipled, side-effectful and Object Oriented Programming as much as possible.  In its place, we will
learn a data centric kind of programming model, more powerful type systems, and controlled mutation with the rust
programming language 

The lessons will cover the following topics:

- Why FP and data oriented programs are the future
- The basics
- Expressions vs. statements
- Modern type systems
- Thinking in terms of data instead of "Objects"
- What side effects are and how to encapsulate them
- Handling errors and nulls
- Imperative vs compostional programming
- Controlling Ownership: Boxes, Reference Counting, and Interior mutation
- Multithreaded Programming
- Advanced types: GAT
- Advanced types: async
- Advanced types: Type Level Programming with PhantomData and State Machines

And we will demonstrate pragmatic rust with many examples:

- An asynchronous http client
- A parser combinator to read graphql
- An async web service with actix
- A python module in rust
- A webassembly module that can be executed in java


## Motivation

Before starting, I wanted to go over why I wanted to create this tutorial.  I have been dabbling in Functional
Programming for about 12 years which was when I first heard about clojure.  At the time, I was feeling frustrated with
Java, but I couldn't really put my finger on _why_ I was frustrated. I had heard Scala being called _Java.next_, so I
bought Odersky's book and started reading it.  After a couple of chapters, I gave up because there were too many
"usually"s for my taste.  Like you "usually" didn't need to add a semi-colon which reminded me too much of Perl (friends
don't let friends code in perl of javascript less than version es6).  But still, Java felt wrong and I couldn't really
articulate why.

Then I [read][-slides]/[watched][-video] Rich Hickey's article about State and how we were doing it all wrong.  It was
an eye-opener.  I felt like I had some Hallelujah moment and a prophet was speaking to me.  I finally could put a finger
on what was bothering me: state management.  As I dug deeper into Functional Programming, I realized there were other
things that were problematic, like handling nulls, or just error handling in general.  Eventually, I came along the rust
language, and realized that you could have a language with disciplined mutation of state and did not have Objects as we
are generally accustomed to.

The result is that rust's programming model is neither fully Functional nor is it Object Oriented. This allows rust to
be more pragmatic and also blazing fast compared to true 100% FP languages or environments while at the same time
avoiding many of OOP's flaws.  Part of the trick in learning rust is understanding its programming model.

So I will talk about Functional Programming in general, and what parts of rust follow this model.

## A brief overview of Functional Programming

It's been several years now that people have been talking about Functional Programming.  Every recent popular language
(with the possible exception of golang) has leaned more towards FP than imperative OOP.  But what **is** Functional
Programming anyway?  FP doesn't really have a standard definition, but it does tend to follow a majority of these
tenets:

- Functions as first class entities
- Anonymous functions (aka lambdas)
- Preference for pure functions
- Expressions over statements
- Functions as control structures
- Control flow as composition of functions over imperative sequencing
- Data is data
- Parametric or ad-hoc polymorphism (generics) over subtyping polymorphism (inheritance)
- Structural (what a data type has) over Nominal (what a data type is) subtyping
- Advanced type systems

We will go into why this style has many innate advantages, but before we go too far let's talk about the elephant in the
room: why learn rust when I'm already competent in Java?

## Why learn rust

Learning a new language is not a trivial effort, so there has to be a risk to reward payoff to do so.  I will assert
that by learning rust you will reap several huge benefits:

- No more NullPointerExceptions
- Advanced type system (type inference, affine types, Phantom types, ADT, GATs)
- Dependency Management that "Just Works" most of the time
- Stand alone executables (who needs containers, or the hair pulling experience of Graal Native?)
- Virtually zero startup time (perfect for lambdas and CLI's)
- Two or three Orders of magnitude superior memory usage (remember, EC2 instances are billed by CPU and RAM)
- Blazing fast (save the planet by reducing energy)
- Deterministic memory consumption and deletion (no garbage collector)
- First class WebAssembly support
- A way to implement behavior of data _after_ a data type has been declared (even if not your data type!!)
- Modern documentation in markdown

The no nulls and dependency management that works should be enough to knock out all the JVM based languages (kotlin and
scala3 both have no nulls also, but the dependency management story is still awful on the JVM).  While rust's type
system is not as advanced as scala3's, it does have a few tricks up its sleeves like affine types and first class rules
for ownership that makes it threadsafe out of the box (for scala3, it doesnt have affine types or ownership, and you
need something like Zio or Cats, and even those are libraries which could still have errors).  I also wanted to stress
something personally important which is the vast performance of rust both in terms of memory usage and CPU utilization.
I will dedicate an Appendix on this to show that as a species, we no longer have the luxury of just shrugging our
shoulders in terms of how much energy we consume on the planet.  To analogize, running slow languages is akin to driving
clunky SUV's that get 10 mpg.

We should be _embracing_ new paradigms rather than trying to knock them down because we either don't want to learn
something new or feel like our existing skillset is obsolete.  The fact of the matter is that technology is _always_
changing and you should accept that about a career in software engineering.

## But I can solve these problems already with Java

This headline above is a common answer that many have when presented with a new way of doing things.  I believe that
many engineers and managers have a lot of the following misconceptions.  Rust is not expressive, because it is too low
level and requires you to "think like the computer does".  Companies are afraid to try it because they think it will be
too hard to find people skilled in it, or it will be too hard to integrate into other existing code. Managers with
existing teams may be afraid of dissent in the team for needing to learn another language that seems niche.  Engineers
may feel that learning a new language is a waste of time, often because they feel problems with their language is
exaggerated or think _all_ languages have the same kinds of problems.  Other engineers feel that problems are due to
lack of discipline and diligence.  And finally, some think that a new language like rust just doesn't have the 3rd party
support needed.

While every paradigm has its pros and cons, we need to ask ourselves if the benefits outweigh the negatives.  So let's
look at each of these claims.

### Productivity suffers

The argument here is that if an engineer is already productive in say Java, what's the benefit to moving to rust (or 
even scala3)?  Afterall, the engineer will be far more productive in a language they have tons of experience with right?

First, let's tackle what we _really_ mean by `productive`.  All too often, engineers and managers especially, have a
very limited view of productivity.  In a narrow mindset, productivity means "how many features can you implement or bugs
can you fix per sprint?".  The narrowness here is all too common and doesn't factor things outside of actual coding
like:

- Writing documentation
- Testing your code
- Time spent debugging and triaging or responding to pager alerts
- Time _thinking_ about the problem 
- Reading other people's code
- Getting your code to actually build (fighting with dependencies, maven, gradle, etc)
- Dealing with infrastructure
- Going to meetings or informal discussions about a feature with engineers, PM's, etc

Writing documentation is both in the small (API documenation) and in the large (design docs).  Testing code means
writing unit tests, test case requirements and doing ad-hoc testinng. Debugging and triaging can take a huge amount of
time, not just in tthe actual debugginng, but creating or updating Jira issue, or looking at APM metrics.  Before we
even start coding we _should_ be thinking about the problem, like is this feature what is wanted or needed?  Or how can
the feature be designed to be extended easily in the future?  Reading code is often more time consuming than writing it!
This is why documentation is so important, not just for code reviews, but for long term understanding.  Who hasn't had
issues with build tools like maven or gradle?  Just setting up infrastructure to run tests and CI/CD through ansible
playbooks, shell scripts, Jenkinsfile etc. can take a long time.  And let's not forget the dreaded meetings (even
informal meetings can take an inordinate amount of time).

As you can see, actually coding is just one small portion of your life as an engineer. Yet, for some reason, too many
people just equate "productivity" with how fast you can release a feature.  We often don't consider the ease of
refactoring code for example.  Languages with less powerful type systems often have issues when refactoring code.
Perhaps you think Java has a powerful type system?  Java's Generics are really not all that powerful (type erasure,
differences between primitves and Object types), You can freely cast willy-nilly, no Abstract Data Types, etc.  This
makes refactoring a little more challenging when you don't have a type system that is itself Turing Complete.

One of the few lessons I remember from my Software Engineering class was that 80% of the time spent on a feature was
after code was launched (maintenance and fixes).

### How do we find people who know rust?

This is probably _the_ most common argument I hear against FP.  It's usually said by managers, but I have heard many
engineers echo this too.  The answer to this is, "you learn it on the job if you can't find new hires".

One of the things I like looking for in any potential candidate is how many languages and frameworks they have actually
used.  It gives me a rough idea of a couple things.  It lets me see they are interested in expanding their knowledge.
It also shows that they know several ways to skin a cat and that they can think of different solutions.

Any engineer worth his salt should be able to pick up Functional Programming principles and rust. The biggest challenge
to learning rust will be learning how lifetimes work, because that is essential to understand how borrowing and
ownership works.  You will beat your head against the compiler for awhile.  But the compiler is doing you a solid.

### Team buy in

This is probably the only argument that I actually buy.  While I believe that all engineers _can_ learn rust and FP, the
truth is that many simply don't want to.  Unless they are forced to by management, many engineers are very happy doing
what they have always done, and will resent having to learn not just a new programming language, but a new way about how
to solve problems.

I will assert that our job as an engineer is about efficiency and robustness.  We _should_ be using paradigms and
concepts that improves those.  And as I mentioned above about productivity, I would hope that this is a good enough
reason.

Except I know it isnt.  Some engineers will have to be dragged kicking and screaming, even if it will benefit them in
the long run.  I've known engineers who practically said they would quit, if they had to learn a revision control system
(yeah....I'm that old).

### Rust is a niche language

I think some people might be excited to learn go, typescript or python, but feel some trepidation, ambivalence, or even
concern if they have to learn rust.  I wanted to present some facts that people might not be aware of if they think that
rust is a 2nd place loser to Go, or it's only popular for low-level programming.

For starters, rust just recently entered the [top 20 in the TIOBE][-tiobe] list.
TIOBE is of course a not so great indicator.  We can instead look at other things

- Rust won the stack overflow developer survey as best language 6 years in a row
- Linus Torvalds recently gave a nod to Rust to be used in kernel (even C++ isn't allowed)
- Microsoft has gone full on board to using rust
- AWS has a growing number of teams using rust (AWS Lambdas/Fargate was written in rust)
- Google just said they started experimenting with putting rust in the Chrome browser to reduce security bugs
- Google's chromebook VM manager is also written in rust
- Facebook joined the Rust Foundation to promote rust (Libre is written mostly in rust)
- Even data scientists are realizing [Python isn't fast enough and starting to use rust][-rust-v-python]

If you are thinking, "well, all these examples are for systems level programs, like web browsers or virtual machine
managers or even operating systems", rust is also making huge head roads into bread and butter things, like http
services.  The actix web server is, according the the TechEmpower benchmarks, the 5th best (only some C++ frameworks
beat it and just barely).  Rust is also making way into Big Data with DataFusion, Apache Arrow or the new influxdb-IOx.

### Exaggerated problems with Java

Rust has been the most loved language on stack overflow for 6 years in a row.  There's a reason for that winning streak.
Once you get over the initial learning curve, you realize that so many problems you have been struggling with in your
current language doesn't even exist in rust.

So let me ask you something, do you think none of this applies to your project?

- My code never gets null pointer exceptions or ArrayIndexOutOfBounds
- I think Checked Exceptions rock!
- I never run into dependency problems when I import libray B and C into my project
- My service never gets Out of Memory errors, or 100% CPU utilization during garbage collection pauses
- I am perfectly fine with my project taking up 500Mb (or more) for a simple service or app
- Size limit in AWS lambda don't bother me, that's what Layers and containers are for
- I never get tired of explaining to users that they need version X of python/JS/JRE to run my app
- I never tire telling people how to get/use docker and explain how to do to `docker run {my-app-image}`
- Users never complain that their docker images use up so much space and constantly have to prune it (eg `rmi`)
- I never wished that a class could represent two or more different things (eg a String or Integer)
- I never wished that I could add behavior to a class I didn't create myself
- I never tire of writing tons of boilerplate for POJOs
- I never have issues (de)serializing my POJOs to JSON
- If I want to write web code, that's what JS is for
- It's awesome to wait 3 seconds for the JVM to spin up my lambda or CLI app
- It's so cool to figure out what breaks each time I try to make a Graal Native image
- Maven is so easy, it does everything I need as long as I know how to configure a plugin
- Gradle is so easy, if there is no plugin, I can just write my own DSL code

And this is just talking about  technical issues with the Java language and JVM infrastructure itself.  I'll save how
Object Oriented Programming failed us as a paradigm in the next section.

### It's just a matter of being diligent and disciplined

Maybe you might be thinking that all the above can't be that big of a deal, after all, look at how many successful
projects were built with Java!  Sure, but that's like saying "hey, we don't need modern construction machines because
look at how ancient Egyptians built the Great Pyramid at Giza!"

Some of these problems actually can't be solved by discipline or diligence, or following SOLID best practices.  For
example, some will point to the maven shade jar as solving Jar hell for the JVM, except that it doesn't.  To be fair, 
rust's dependency management can also fail, but typically only in three scenarios:

- You are dynamically linking in a library (in which case, no name mangling can happen by the compiler)
- You have `const static` vars with different values in different crates
- You are (de)serializing objects between 2 different version crates (sharing data across crates can fail)

The ways that JVM dependencies can fail are numerous, especially because Java loves static classes so much where in rust
it's typically frowned on.

Another example is the inability of Java to get rid of NPE's, or for backwards compatibility, to get rid of
ArrayOutOfBounds.  If you think you can be disciplined to catch these, then there are tons of expert C++ programmers who
still make memory access errors failures who can convince you otherwise.  And they too heard "it's just a matter of
being disciplined".  That's why kotlin and scala3 decided to make it explicit if a type is nullable or not.  And rust
only has a `null` type if you do `unsafe` programming in order to interop with C libraries.

### Rust is lacking a 3rd party lib I need

2 or 3 years ago, I would have agreed that this was a valid concern.  But now, rust has just about everything covered.
Some libraries are actually best in class, for example rust's `serde` crate.  Rust also has the fast implementation of
a regex engine anywhere (that powers the `rg` cli tool).  If you want to create a web service, there's actix, warp and
rocket.  If you want an http client, there's reqwest.  AWS support?  Currently it's in preview state, but AWS does have
a rust sdk and a lambda runtime.

And if somehow, rust is missing a 3rd party library you need, is it something you can incrementally work on?  For
example, maybe there's an SDK that exists, but not for rust.  You only need a small part of the library, maybe to make a
REST call to get data.  All you really need to do know is the data for the POST, and what the data looks like coming
back and you can create your own enum/struct to model the data over time.

This being said, rust is nowhere near as full of libraries as Java, python, javascript or go.  You probably will run
into some scenarios where you wish rust had a crate to do it for you.  I will argue though that creating your own
libraries and publishing them upstream (or contributing to existing ones) is a worthwhile effort.  Being able to control
your own library (and fix CVE's) is nothing to take lightly.

### It won't fit with existing code

In this day and age, that argument doesn't fly so well.  For starters, so many things we do are basically REST or
graphql calls which return JSON.  In fact, this movement is so strong, that the Java architects are moving towards a
more data oriented style of programming.  But, you say all of our libraries are written in some JVM language, how can we
get rust to interop with it?

There are a couple of answers, if you don't want to rewrite.  And rewriting IS an option.  Google rewrites their own
services every few years.  I always laugh when someone tells me it is "stupid" to reinvent the wheel.  I always tell
them, if we didnt reinvent the wheel, we wouldnt have trains (metal wheels) or cars (rubber wheels).  Google rewrites
their services because it allows them to make improvements learned from experience, and also to give new engineers a
chance to truly understand what the system is really doing.  I am fond of a quote by Richard Feynman, the renowned
physicist.  He said, "what I can not build, I do not understand".

But if you really don't want to rewrite, and you have a library and not a data exchange format you can use (gRPC, Json,
etc) then java is slowly getting an alternative.  It's called WebAssembly.  WebAssembly is in some ways, "java done
right".  One of the reasons people are interested in webassembly is because it brings to within near native performance,
but most importantly it can be a universal bytecode.  Where Java and .Net have their own bytecode, webassembly was
designed to be more portable and lower level in some ways.  As a result, many languages now have experimental support
for webassembly.  This means for example, that you could have a library written in python, compiled to webaseembly, and
used in a Java app.

This is what that GraalVM's "polyglot" feature and embedding is all about.  Rust and C++ were really the two languages
that the W3c WebAssembly group used as first class citizens.  In other words, c(++) and rust have the best support for
generating webassembly code.  Many of the wasm runtimes are also built in rust (eg, WASI )

## How Java's OOP failed us

So, let's keep an open mind and consider that we should be doing something different.  I will argue that OOP has failed
us by making things more complicated than they needed to be.  Or rather, I should say OOP + Imperative programming
(which is pretty much what everyone does).

If you don't believe me, the [Java architects themselves have explicitly stated][-java-blog] they are moving towards a
more data-oriented style of programming.  This can be seen in several new features they have made that all borrow from
FP:

- Record types: immutable data containers
- Value types (Project Valhalla): goes hand in hand with records, and is a huge performance boost
- Sealed types: Enums on steroids that allow you to specify a limited set of related types
- Pattern matching: Only works with immutable types but allows destructuring and matching of data

The java architects have also gone on the record admitting other failures.  Optional was a half baked way to try to
manage the disaster of null. Checked Exceptions sounded nice in theory, but failed in practice. And getting the wrong 
defaults for many things.

Perhaps the biggest failing of Java is just the concept of OOP itself which Java embraced 100%.  You can't even create a
standalone function without embedding it into a class.  Then there's the whole primitives/arrays vs Object divide (which
Project Valhalla should resolve). But why did OOP fail, and why is Java, the mid 90s champion of OOP, deciding to move
on? At least, as much as it can since it is hobbled by legacy support and will probably never be able to completely
divorce itself.

OOP and undisciplined imperative programming failed us in several ways

- Side effects were paid little thought, and only pure FP languages with advanced type systems can even track side effects
- Defaulted to mutable rather than immutable data
- Standalone functions either had no representation at all (java <=7>) or are hobbled (java8+)
- Imperative programming assumes synchronous programming and a sequential style
- Exception handling was handled in its own little world, rather than being treated as a type

### Side effects are an afterthought

Side effects are so much an afterthought, that some engineers don't even know what the term means unless they have
learned a FP language!  I wrote an article on Dev Zone and talked about misconceptions around Functional Programming.  I
saw more than one comment say that "if my code intended to change an outside variable or write to a file, that's not a
side effect since my purpose was to do that!".  Unfortunately, that's the colloquial definition of a side effect: an
unintended consequence of some other action.  

In Programming Language Theory, side effect has a much more precise definition.  A side effect is when a function either
affects the "outside world" (anything out of scope of the function itself or the arguments passed into the function) or
the result is affected by the outside world.  For example, any kind of IO operation like reading from or writing to a
file, a network socket or even the console is a side effect.  Another more insidious side effect is time.  Time is an
implicit factor of state (state is defined as "value of an object at a point in time"), and it is insidious because it
is hidden.  Race conditions happen because of time.  A function could fail because the file that used to exist changed a
millisecond ago isn't the same _now_.

Side effectful functions therefore are not _pure_.  A pure function is a function that obeys a couple of laws:

- It can't have side effects
- Therefore, given the same arguments, it will always give you the same answer
- As a consequence of the above, a pure function is immune to time (an implicit argument in side effecting functions)
- By implication, a pure function must return something or the answer is never known

Pure functions therefore are immune to the outside world and also do not disturb the outside world.  The only way to
interact with a pure function is to give it inputs and recieve an output.  Because the only thing a pure function cares
about are the arguments you pass in, they are much easier to reason about.  You don't have to think about:

```
- When the function is being called
- If some service is down
- If some file or filesystem exists
- A race condition with some other function
- Mutating an argument that was passed in that another function may need
```

And many other things.  You may be arguing that OOP can have pure functions, and you would be correct.  However, the
very notion of OOPS is that it combines data (really state) with behavior.  The methods of a class are driven by the
current state of the object, or update the state of the object.  It's practically designed to be impure.  Purity in a
function makes it very easy to reason about, whereas in OOP you always have to wonder what is changing under the hood.

As an example of a pure function, think about addition.  You give addition 2 numbers, and you get back a number.
Addition doesn't mutate the numbers you put in.  It doesn't send the answer to a service which might return a 500.  It
doesn't save the result to a file.  It doesn't even write the answer to STDOUT.  One could argue that if we don't have
_some_ side effect, then how would we ever know what the answer was?

This is entirely correct.  If you do a pure computation, you would never be able to know what the compued result was!
So at some point, you **must** do side effects.  Functional languages try to restrict where the side effects happens 
and some languages can even encode the fact they are effectful in their type system.  Rust only has partial support for
this, as for example by having asynchronous effects modeled directly in the language.  The lifetime and ownership rules
also accounts for the effect of mutation.  However, rust has no way to encode effects in a generic sense, like IO in 
haskell or the Cats library (eg, writing to a file, reading from a network socket, etc).

### Immutability

Going hand in hand with side effects is immutability.  One extemely popular language doesn't even _have_ the concept of
immutability (cough, python, hack...and yes, I know eg an `int` is immutable, but you can not declare a variable itself
as unchanging since there is nothing like `final`, `const`, or `val` in python).  But why should we care about
immutability? Why has it become so important in modern language design?

It comes down to two things really:

- Being able to reason about code when a variable cant change
- Being safe for sharing across threads

It's even unfortunate that we use the term _variable_ to denote "things that hold a value".  The problem is that while
variable can mean "something that can be substitued for", we don't have a way to distinguish _how many times_ you can
perform the substitution (only on initialization, or even afterwards?).  Some languages might differentiate them with a
`const` or `final` modifier, or perhaps distinguishing between `val` and `var` (kotlin and scala do this), but we still
call them _variables_ unfortunately.  Or even more confusingly "constant variable" or "immutable variable".  How can
something be constant or immutable, and also _variable_?  

This is an unfortunate legacy of programming.  Even the age old lisp called them `vars` or `defs`, but at least lisp
also separated out the idea of the _name_ of variable (a `Symbol`) from the thing the symbol referred to (the data it
represented).  

This sleight of hand is so second nature to us, that we just think of the name of a variable as equivalent to its value.
Unfortunately, we don't really stop to think about all the machinery going on under the hood (unless you are a C(++) or
rust programmer).  Consider the following in java:

```java
var name = "Sean";
```

Here, `name` has a string the value of which is "Sean".  But what is going on in the computer?  The variable `name` is
just a reference, which is really a kind of pointer to a memory address.  This memory holds a contiguous chunk of bytes,
which when decoded by utf16 comes out as "Sean".  What would happen if I then do this?

```java
name = "John";
```

Java wouldn't care.  The compiler will go, oh ok, the engineer wants to change the _value_ that `name` points to.  But
what does that even mean?  There are two possibilities.  The language could either:

1. Change `name` to a different location in memory, where its value is "John"
2. Erase the memory that `name` points to, and put "John" there

Since `name` has a String type, and Strings are immutable in Java, Java does the first option. But what if the type
wasn't immutable?  Now things get messy.  Now, the compiler will write code that might do the 2nd option.  

```java
import java.util.Set;

class Person {
    String name;
    Integer age;
    Set<String> languages;

    public Person(String name, Integer age) {
        this.name = name;
        this.age = ahe;
    }

    public static void main(String... args) {
        var println = (Sring x) -> System.out.println(x)
        // How immutable data works
        var s = "Sean";
        var s2 = s;
        s == s2; // true.  s and s2 are the same object
        println(s.hashCode());
        println(s2.hashCode());  // prints the same hash code for each
        s = "John";
        println(s2);  // still prints "Sean"
        s.hashCode(); // is now different
        s == s2;  // this is now false since we changed where s is pointing to

        // how mutable data works
        var p = new Person("Sean", 49);
        var p2 = p;

        p == p2; // true
        println(p.name); // "Sean
        println(p2.name); // also "Sean"
        p.name = "Alex";  // What do you think p2.name is?
        println(p2.name)
        p == p2;  // This is still true.
    }
}
```

A source of never ending confusion is that in the example above, when you change `p.name`, since `p2` is pointing to the
same memory, although p2.name _does_ change, their hashcodes are still equal.  This _might_ be what you want, but more
often than not, most programmers assume that p2 is just a copy of the data, and thus shouldn't change.  So already,
immutable is more intuitive!

And we aren't even doing multithreaded programming yet!  If you have some object, and the values inside the object can
change over time, you now have to keep track of who could change the data, and when, evem in a single threaded
environment.  If you forgot that that some other component had access to the object, or even part of the object, then
you might be surprised when the answer doesn't come out the way you think.

How about rust?  Is it different?

[-video]: https://www.infoq.com/presentations/Are-We-There-Yet-Rich-Hickey/
[-slides]: http://wiki.jvmlangsummit.com/images/a/ab/HickeyJVMSummit2009.pdf
[-eta]: https://docs.scala-lang.org/scala3/book/fun-eta-expansion.html
[-java-blog]: https://blogs.oracle.com/javamagazine/post/what-are-they-buildingand-why-6-questions-for-the-top-java-architects
[-rust-v-pythonn]: https://www.nature.com/articles/d41586-020-03382-2
[-tiobe]: https://www.infoq.com/news/2020/06/rust-top-20-language/

In [ ]:
use std::collections::HashSet;

struct Person {
  pub name: String,
  pub age: u64,
  pub languages: HashSet<String>
}

fn test() {
  let person = Person {
      name: "Sean".into(),
      age: 49,
      languages: HashSet::new()
  };

  println!("Name of person = {}", person.name);
  //person.name = "John".into();  // compiler error here, since we didn't declare person to be mut

  let person2 = person;
  println!("name of person2 = {}", person2.name);
  // println!("name of persion = {}", person.name); // cant do this because the data in person is now owned by person2
  // This change is very counterintuitive because we do it in other langauges, and yet, as shown in the Java example,
  // it's actually a source of many bugs.  And I didn't even give an example of shared access to a List<T>
}

test();

So we can see that kotlin and scala still follow the same rules, except that you have to explicitly state whether a
variable is mutable or immutable right at declaration.  So you have to think right at the begining what you want.  In
java, it defaults to mutable, so unless you really know ahead of time, you might forget to add `final`.

It's bad enough in a single threaded environment where any field with a setter (public or private) can change your data.
Now imagine multiple threads having access.  But wait!! Isn't `private` supposed to help me by limiting who can set the
value?  The problem doesn't go away with private setters.  Because the problem isn't just "who" changes the data, but
"when".  Again, time is your enemy because it is almost always a hidden argument in side effectful and mutable programs.
Also, if data is immutable, the compiler can perform some enhancements and guarantees, because it can reason that _no
one_ can change the data.

Rust, learning from the mistakes of so many languages, made immutable variables the default, and you had to use a `mut`
keyword to tell the compiler "the value in memory can change in place".  Kotlin, scala and javascript decided that there
would be no default, and you had to specify if it was either mutable (`var`) or immutable (`val`).  Java, being an
old-school language made mutability the default, and you have to tell java you want immutability by using the `final`
modifier.  Even the Java architects admit this was a mistake.

Mutability does have one advantage though: it's faster.  Changing the data in place is faster than copying the entire
dataset.  Imagine an array with tens of thousands of elements.  You really don't want to copy the whole thing when you
change a single element.  So, mutability still has its place, but be wary of it.  

Unless you are using rust, whose compiler is virtually unique in being able to guarantee us that only one owner can ever
modify a value at one time, even in a multithreaded environment, prefer immutability unless performance is critical.
Performance and safety: that's why rust is so loved.

## Lack of top level functions

Java was designed to be a 100% OOP language.  Unlike some of its contemporaries like C++ or python which were mixed
paradigm languages, Java went hog-wild on OOP and effectively made it the _only_ way to program.  Java didn't realize
the error of its ways for many years.

Next to Generics (released in Java 5), the biggest change to the platform came with Java 8's lambdas.  But even lambdas
are somewhat crippled.  Why?  Lambdas must still be embedded inside a class.  There is no way to define a function that
exists on its own.  It must belong to a class.  However, at least lambdas meant that they could be stores in variables,
passed in as arguments, or returned from functions.

But there's actually a deeper problem lurking.  In java, because there are _only_ classes, there really is no such thing
as a _function_.  Saying "function" in Java is technically incorrect, because there are only _methods_.  Why does that
matter? Think about it this way: in the JVM, methods have an implicit object passed to them (named `this`).  What does
_this_ refer to when you have top level methods that don't exist in a class? 

On the contrary, in rust, functions are first class citizens that are defined at the top level of a file.  They can also
be declared in `impl` blocks, either as the definition/declaration for Traits, or as associated functions for `struct`s
and `enum`s.  They can not however, be bested inside other functions as is sometimes seen done in other languages.

This leads to a slightly confusing part with rust in regards to higher order functions, and there will be a separate
chapter talking about the different kinds of functions in rust.  But to give you a taste, consider the code below: 

In [2]:
// This is a function
fn as_lower(s: &String) -> String {
  s.to_lowercase()
}

fn test() {
  // this is a closure (aka lambda).  Its type is different from `as_lower`
  let lowered = | name: &str | {
    name.to_lowercase()
  };

  let names = vec!["Sean", "John", "Ron"].into_iter().map(lowered); 
  for name in names {
    println!("{}", name);
  }
}

test();

sean
john
ron


## Imperative assumes synchronicity

What's worse than OOP?  OOP with imperative programming.  Unfortunately, we all do it, including me, but I hope that I
can show you over the course of this tutorial that there is another way.  But first, some disclaimers.  Is imperative
programming always bad, or innately bad?  One scenario where you will want to do mutable imperative programming is when
performance is critical.  While you may have heard some FP evangelists claim that FP is faster because it allows better
parallelism, they usually neglect to mention that immutable persistent data structures ARE less performant, even with
data structure sharing.

Often, we can't even share data structures and we just need to modify a single field in a data type.  In this scenario,
you have to create a brand new object.  This can get expensive.  If you wonder why it is expensive, I will need to get
into how the heap, stack, and cache work, and explain the latencies that come with each of them.

But for now, let's concentrate on imperative sequencing.  This is so common, that we don't even stop to think that there
is another possible way to program...at least until you start needing to do asynchronous programs.  Consider the
following:

```java
var response = client.httpGet(someUrl);
if (reponse.status != 200) {
    log.error("Failed to get resource");
    // do something with the error
} else {
    log.debug("Getting body from response");
    SomeClass.doWork(response.getBody());
}
```

Pretty cut and dry right?  First you make an http request, them you check the status and either handle an error, or do
something with the response that was returned.  Easy-peasy right?

Unless, that `httpGet` call takes forever.  So ok, you put in a timeout.  But then you have 3 things to consider:

- An actual failure (ie, a 404 or 500) from the server
- A sucessful response
- Or no response (within the timeframe)

Or worse, what if your code times out, but was eventually successful?  On a GET this is probably not a big deal, but
what if it was successful, and updated a database entry or modified a file?

Ok, maybe it's not that simple.  But let's think even more basic.  What is this imperative code _really_ doing at its
most basic essence?  To really understand this question, you have to consider that all code is one of two things:

- A statement
- An expression

These are mutually exclusive choices.  A statement doesn't return a value.  It is either an assignment, or a side
effect.  

In [ ]:
let x = 100;  // a statement
println!("x = {}", x);  // this is actually an expression that returns Unit
let y = x * 2;  // an expression
let z = if y > 12 {  //yet another expression
    "more than a dozen"
} else {
    "less than a dozen"
};

// this is invalid rust
// while let name = getNextName() {  the assignment operator returns nothing not even Unit
//    name = getNextName()
// }

## Expressions over statements

FP focused languages are expression oriented.  Almost all constructs are expressions.  Java, being an imperative
language has a strong preference for statements.  That is why it is called an if statement or try/catch statement in
Java, because if and try blocks don't return values.  You have to do assigments inside the block.  But then Java is
weird, and makes assignments return the value of the assignment.

So what is really going on in a block of imperative code?  You make a call that returns a value.  You then use the
returned value in a later call.

```java
var accessKey = Manager.getResource();
var data = client.makeRequest(accessKey);
var purchaseEvents = data.events.stream().filter(evt -> evt.count > 100).collect(Collectors.asLis());
Validator.ensurePurchaseQuantity(purchaseEvents);
```

But, you could have just as easily done this

```java
Validator.ensurePurchaseQuantity(
    client.makeRequest(
        Manager.getResource()
    ).events.stream().filter(evt -> evt.count > 100).collect()
)
```

This gets a little hard to read, because composition works from the inside out (eg, in f(g(x)) you solve for g(x), and
then the output of it is used to solve for f).  What if we could rewrite it in a different way?  If you have used Unix
at all, you are probably familiar with the pipe operator.  The pipe operator "pipes" the output of the command to the
left, as the input to the command to the right.  It's kinf of an inverse of a mathematical composition.  So in scala,
you can create not only your own operator like pipe (which in the example we will write as |>), but you can also call
them in infix mode.  In kotlin, you have a limited number of operators you can override, but you can also call functions
in an infix mode.  So imagine that we have an operator in scala |>, like the | operator from unix

```scala
Manager.getResource() 
  |> client.makeRequest 
  |> { data -> data.events.filter { evt -> evt.count > 100} }
  |> Validator.ensurePurchaseQuantity
```

At this point, you might be asking yourself, "so what?  What's the difference?".  There are a couple of differences
between the two styles.  The biggest is that in the functional style, a program is nothing more than the composition of
functions.  This has profound ramifications and requires a change in thinking, and designing both your problem and your
code.  But by doing so, it also eliminates many common bugs which I will go over throughout the tutorial.

It also paves the way for solving programs which are asynchronous.  If you have ever done any amount of javascript with
Promises, you may have noticed (and cursed) at how _contagious_ Promises are.  What do I mean by contagious?  Once you
have something that returns a Promise, now your whole computational chain must become a Promise.  You can't have some
synchronouse code "outside" use the value of the Promise, unless you made that code part of the Promise chain.  I often
saw incorrect code like this by newbie Javascript devs:


```javascript
let someNumber = 0
let promised = somePromiseFunction()
  .then(p => {
      let NewVal = getFromDb(p) // getFromDb() may take awhile
      if (newVal > 50) {
          someNumber = 100
      } else {
          someNumber = newVal
      }
  })
if (someNumber == 100) {
    console.log("we were successful")
    doSomething(someNumber)
}

// This is how you _should_ do it
let asyncVal = somePromiseFunction()
  .then(p => {
      let NewVal = getFromDb(p) // getFromDb() may take awhile
      if (newVal > 50) {
          return 100
      } else {
          return newVal
      }
  })
  .then(latestVal => {
      if (latestVal == 100) {
          console.log("we were successful")
          return doSomething(latestVal)
      } else {
          return null
      }
  })
```

This code will almost never print "we were successful", because the programmer assumed that the Promise chain ran
synchronously in the same order as the code itself.  But what happens is javascipr's main event loop will kick off the
`somePromiseChain` promise, but then immediately get to the line where checks what the value of `someNumber` is.  Unless
`getFromDb` is really really fast, this will almost certainly fail.  This is why Javascript ES6 came with async/await
syntax.

```javascript
async function foo() {
    let someNumber = 0
    let promised = await somePromiseFunction()  // this blocks now
    if (someNumber == 100) {
        console.log("we were successful")
    }
}

(await foo())
```

However, async/await is really just syntax sugar around Promises, and Promises have quite a few potential pitfalls.  The
point here though is imperative programming assumes a synchronous world which may not be valid.  When you have a program
which follows these rules"

- functions are pure
- functions are well typed

If the above is true, you can do something very strange and powerful when you design your program as a composition of
functions that are well typed.  You can "embed" extra information in your function and "carry" them along as you
traverse your computational graph.

You can think of programming as the composition of functions as a graph.  In a single threaded synchronous program, it
is a linear single node per vertex chain of transformations.  In a multithreaded or asynchronous application, you can
have multiple nodes off of one vertex (though you now cant think of the edge as the output-input edge, but now possibly
also as a "spawned a new thread/task/coroutine" sub-graph)

But we will go into this in more detail as the book progresses.  For now, hang tight, and think about functions as
compositions of functions that create a graph and that doing this can make your multithreaded and asynchronous programs
easier to reason about.